# Taux de dissolution en fonction des Process, des MP et du PA

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# Read data
from pathlib import Path
import wget
from zipfile import ZipFile

## docker path
xpath ='data'

## path docker
#xpath = 'work/data'

data_file = xpath+'/app_pca_data.zip'

fileObj = Path(data_file)
# if not fileObj.is_file():
#     url_zip = 'http://droppy.scale-eu.sanofi.com/$/8MhWS'
#     wget.download(url_zip, 'data/app_pca_data.zip') 
    
with ZipFile(data_file, 'r') as zipObj:
    zipObj.extractall(xpath)

# df_process= pd.read_csv("data/disso_release_pelliculage.csv")
# df_mp= pd.read_csv("data/dronedarone_mp_v2.csv")
# df_pa= pd.read_csv("data/dronedarone_pa.csv")

#Doker
df_process= pd.read_csv(xpath+"/disso_release_pelliculage.csv")
df_mp= pd.read_csv(xpath+"/dronedarone_mp_v2.csv")
df_pa= pd.read_csv(xpath+"/dronedarone_pa.csv")


# Dataset

In [ ]:

features_temp = ['Temps_de_melange','Temps_de_mouillage','Temps_de_granulation', 'Temps_de_sechage']
c0 = list(df_process.columns)
features_num =c0.copy()
for x in features_temp:
    features_num.remove(x)
features_num.remove('Batch')

# transform time to seconde
def xsecond(myTime):
    s = myTime.hour*3600 + myTime.minute*60+myTime.second
    return s

df_process2 = df_process.copy()
#Format time
for c in features_temp:
    df_process2[c] = pd.to_datetime(df_process2[c], unit='d', origin='1899-12-30')
    df_process2[c] = [xsecond(t) for t in df_process2[c]]
    
# remove col
col_toDrop = [
    'Pelliculage',
    'Puissance_GRANULATION',
    'Puissance_MELANGE',
    'Puissance_MOUILLAGE',
    'Temperature_enceinte_GRANULATION',
    'Temperature_enceinte_MELANGE',
    'Temperature_enceinte_MOUILLAGE',
]
df_process2.drop(col_toDrop, axis =1, inplace = True) 

## supprimer n premières lignes
n = 115
df_process3 = df_process2[n:].copy()

# fillna
#bfill: use next valid observation to fill gap.
df_process3 ['duree_pelliculage'] = df_process3 ['duree_pelliculage'].fillna(method='bfill')
df_process3.set_index('Batch', inplace = True)

df_process_final = df_process3.copy()



In [ ]:

df_mp.set_index('lot',inplace = True)
#df_mp2 = df_mp.drop_duplicates()
df_mp2 = df_mp.reset_index().drop_duplicates(subset=['lot'])
df_mp = df_mp2.set_index('lot').copy()
df_mp.drop('pf', axis=1, inplace =True )
df_mp.dropna(axis=0, how='all', inplace = True)
df_mp.dropna(axis=1, how='all',inplace = True)
df_mp = df_mp.fillna(method='bfill')

MP_toDrop = ['AMIDON DE MAIS PE/NF - RESULTAT DGAT/g',
             'AMIDON DE MAIS PE/NF - RESULTAT DMLT/g',
              'HYPROMELLOSE 6MPAS PE/USP/JP - RESULTAT DGAT/g',
              'HYPROMELLOSE 6MPAS PE/USP/JP - RESULTAT DMLT/g',
              'LACTOSE MONOHYDRATE PE/NF - RESULTAT DGAT/g',
              'LACTOSE MONOHYDRATE PE/NF - RESULTAT DMLT/g',
             'MAGNESIUM STEARATE VEG PE/NF - RESULTAT DGAT/g',
             'MAGNESIUM STEARATE VEG PE/NF - RESULTAT DMLT/g',
]
df_mp.drop(MP_toDrop, axis =1, inplace = True)
df_mp_final = df_mp.copy()


In [ ]:

df_pa.set_index('lot',inplace = True)
# remove na
df_pa1 = df_pa[['Granulometrie: d(50)-MEAN','Granulometrie: d(90)-MEAN']]
df_pa1 = df_pa1[~df_pa1['Granulometrie: d(50)-MEAN'].isna()]
df_pa_final=df_pa1.copy()


In [ ]:
df = df_process_final.join(df_mp_final)
df.drop(df[df['HYPROMELLOSE 6MPAS PE/USP/JP - TENEUR EN PRODUIT DOSE 2'].isna()].index, inplace = True)
# join with pa
df = df.join(df_pa_final)
#df = df_process_final.join(df_pa_final)
df = df[~df['Granulometrie: d(50)-MEAN'].isna()]
df.info()

# Taux de la variance expliquée 

In [ ]:
import numpy
import scipy

In [ ]:
X = df.drop('RELEASE', axis =1)
y = df ['RELEASE'].copy()
y_release = df ['RELEASE'].copy()

n=X.shape[0]
p=X.shape[1]

#classe pour standardisation
from sklearn.preprocessing import StandardScaler
#instanciation
sc = StandardScaler()
#transformation–centrage-réduction
Z = sc.fit_transform(X)

#classe pour l'ACP
from sklearn.decomposition import PCA
#instanciation
acp = PCA(svd_solver='full')

#calculs
coord = acp.fit_transform(Z)

#valeur corrigée
eigval = (n-1)/n*acp.explained_variance_

#racine carrée des valeurs propres
sqrt_eigval = np.sqrt(eigval)

#proportion de variance expliquée
var_ratio = acp.explained_variance_ratio_
#np.round(var_ratio,2)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = fig = make_subplots(rows=1, cols=2)
# Create scatter trace of text labels
fig.add_trace(
    go.Scatter(x=np.insert(np.arange(1,p+1),0,0), 
                y=np.insert(np.cumsum(acp.explained_variance_ratio_),0,0),
                mode='lines+markers',                   
), row=1, col=1)
fig.add_trace(
    go.Bar(x=np.insert(np.arange(1,p+1),0,0), 
                y=np.insert(acp.explained_variance_ratio_,0,0),
                                   
),row=1, col=2)



#Title
fig.update_layout(title='PCA: Variance expliquée')
fig.update_layout(title_x=0.5)
# Set figure size
#fig.update_layout(width=600, height=600)
fig.update_layout(showlegend=False)
fig.update_xaxes(title="Nombre de dimension")
fig.show()


In [ ]:
import ipywidgets as widgets

wax1 = widgets.Dropdown(
            options= [*range(0, len(X.columns), 1)],
            value=0,
            description='Axe 1:',
            disabled=False,
)

wax2 = widgets.Dropdown(
            options= [*range(0, len(X.columns), 1)],
            value=1,
            description='Axe 2:',
            disabled=False,
)


In [ ]:
from ipywidgets import interact
def tx(ax1,ax2):
    tx1 = var_ratio[ax1]
    tx2 = var_ratio[ax2]
    tx_sum = tx1 + tx2 
    tx_cumul = np.cumsum(var_ratio[:(ax2+1)])[ax2]
    return print(), print('Taux Axe1: {:.2f}%'.format(tx1)), print('Taux Axe2: {:.2f}%'.format(tx2)), print('Axe1+Axe2: {:.2f}%'.format(tx_sum)), print('Taux cumulé depuis Axe1: {:.2f}%'.format(tx_cumul)) 

interact (tx, ax1 = wax1, ax2=wax2)

# Explication des axes 

### Selection des axes

In [ ]:
#corrélation des variables avec les axes
corvar = np.zeros((p,p))
for k in range(p):
    corvar[:,k] = acp.components_[k,:] * sqrt_eigval[k]


import ipywidgets as widgets
from ipywidgets import interact

limite1 = widgets.FloatSlider(min=0,max=1,step=0.1, value =0.6)
limite2 = widgets.FloatSlider(min=0,max=1,step=0.1, value =0.6)


def correl_axes(ax1,ax2,l1,l2): 
    df_covar = pd.DataFrame({'id':X.columns,'COR_AXE1':corvar[:,ax1],'COR_AXE2':corvar[:,ax2]})
    c1 = df_covar [['id','COR_AXE1']] [df_covar['COR_AXE1'].abs()>l1].sort_values(by = 'COR_AXE1', ascending=False)
    c2 = df_covar [['id','COR_AXE2']][df_covar['COR_AXE2'].abs()>l2].sort_values(by = 'COR_AXE2', ascending=False)
    return display(c1) , display(c2)

interact(correl_axes, ax1=wax1, ax2=wax2,l1=limite1, l2 = limite2);

In [ ]:
idx = pd.DataFrame(X.columns)
idx.columns = ['var']
df_corvar_all =pd.DataFrame(corvar)
df_corvar_all2 = idx.join(df_corvar_all).set_index('var')

### Selection du paramètre

In [ ]:
dd_param = widgets.Dropdown(
    options=X.columns,
    description='Paramètre:',
)

limite = widgets.FloatSlider(min=0,max=1,step=0.1, value = 0.3)


def correl_axes_forVar(f,l): 
    c = pd.DataFrame(df_corvar_all2.loc [f,:])
    c=c[c.abs()>l].dropna()
    return display(c)

interact(correl_axes_forVar, f = dd_param, l = limite)


# Cercle de Corrélation des paramètres

In [ ]:
y_release = y
vsQuanti = pd.DataFrame(y_release)

#corrélation avec les axes factoriels
corSupp = np.zeros((1,p))
for k in range(p):
    for j in range(vsQuanti.shape[1]):
        corSupp[j,k] = np.corrcoef(vsQuanti.iloc[:,j],coord[:,k])[0,1]
        
import plotly.graph_objects as go

c_varSup= widgets.Checkbox(
    value=True,
    description='Disso 30mn',
    disabled=False,
    indent=False
)

def graph_cercle(ax1, ax2, varSup):
    fig = go.Figure()
    
    # Create scatter trace of text labels
    fig.add_trace(
        go.Scatter(x=df_corvar_all2[ax1], 
                    y=df_corvar_all2[ax2],
                    mode='markers',
                   # name='markers',
                   text=df_corvar_all2.index
    ))
    if varSup:
        fig.add_trace(
            go.Scatter(x=pd.DataFrame(corSupp)[ax1], 
                        y=pd.DataFrame(corSupp)[ax2],
                        mode='markers',
                        name='Disso 30mn',
                        marker=dict(
                            color='red',
                            size=20,
        #                     line=dict(
        #                         color='MediumPurple',
        #                         width=12
        #                    ),
                        ),
                        marker_symbol ='x'

    ))
    fig.add_trace(
        go.Scatter(x=[-1,1], 
                    y=[0,0],
                    mode='lines',
                    line_color = 'rgb(192,192,192)'

    ))
    fig.add_trace(
        go.Scatter(x=[0,0], 
                    y=[-1,1],
                    mode='lines',
                    line_color = 'rgb(192,192,192)'

    ))


    # Set axes properties
    fig.update_xaxes(range=[-1, 1], zeroline=False)
    fig.update_yaxes(range=[-1, 1])
    # Add circles
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=-1, y0=1, x1=1, y1=-1,
        line_color="LightSeaGreen",
    )
    # Add circles
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=-0.5, y0=0.5, x1=0.5, y1=-0.5,
        line_color="LightSeaGreen",
        fillcolor="PaleTurquoise",
        opacity=0.2,
    )
    #Title
    fig.update_layout(title='Cercle de Corrélation')
    fig.update_layout(title_x=0.5)
    # Set figure size
    fig.update_layout(width=500, height=500)
    fig.update_layout(showlegend=False)
    fig.show()
    
interact(graph_cercle, ax1=wax1, ax2=wax2, varSup=c_varSup)

### Correlation de disso 30mn avec les axes

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

limite = widgets.FloatSlider(min=0,max=1,step=0.05, value = 0.10)

corSupp2 = pd.DataFrame(corSupp).T

def correl_release_axes(l):
    c3 = corSupp2[~corSupp2[corSupp2.abs()>l].isna()].dropna()

    return display(c3) 

interact(correl_release_axes, l=limite)

# Positionnement des lots en fonction des Taux de Disso 30mn

In [ ]:
var = 'RELEASE'




seuil1 = widgets.FloatSlider(min=30,max=52,step=1, value = 30)
seuil2 = widgets.FloatSlider(min=30,max=52,step=1, value = 42)

#batch = pd.DataFrame(X.index)
X2b = pd.DataFrame(y_release).copy()
X2b.reset_index(inplace = True)
#X2 = X2.join(batch)

def graph_release (s1,s2,ax1,ax2):
    NewVar = var+'_cat'
    if s1<s2:
        if s1 == 30:
            X2b[NewVar] = ['sup'+ str(s2) if x >s2 else 'inf'+ str(s2) for x in X2b[var]]
        else:
            X2b[NewVar] = ['sup'+ str(s2) if x > s2 else 'inf'+ str(s1) if x < s1 else 'neutre'for x in X2b[var]]
        coord2 = pd.DataFrame(coord)
        coord2 = coord2.join(X2b)

        import plotly.express as px
        fig = px.scatter(coord2, x=ax1, y=ax2, color=NewVar, hover_name='Batch')
        fig.add_trace(go.Scatter(x=[-8,8], y=[0,0], mode='lines', line_color = 'rgb(192,192,192)',showlegend=False))
        fig.add_trace(go.Scatter(x=[0,0], y=[-8,8], mode='lines', line_color = 'rgb(192,192,192)',showlegend=False))
        fig.update_xaxes(range=[-8,8])
        fig.update_yaxes(range=[-8,8])

        fig.update_layout(newshape_line=dict(color =('red')))

        fig.update_layout(title='Taux Disso 30mn')
        fig.update_layout(title_x=0.5)
        fig.show()
    else:
        print('s1 doit inféririeur à s2')
    return 

interact(graph_release,s1=seuil1, s2 = seuil2, ax1 = wax1, ax2=wax2)

# Postionnement des lots en fonction de l'année

In [ ]:
X2 = X.reset_index().copy()

X2 [var] = [2015 if x[0:2] == "5A" else 
              2016 if x[0:2] == "6A" else
              2017 if x[0:2] == "7A" else
              2018 if x[0:2] == "8A" else
              2019 if x[0:2] == "9A" else
              2020 if x[0:2] == "AA" else 0
              for x in X2['Batch']
             ]


wax1 = widgets.Dropdown(
            options= [*range(0, len(X.columns), 1)],
            value=0,
            description='Axe 1:',
            disabled=False,
)

wax2 = widgets.Dropdown(
            options= [*range(0, len(X.columns), 1)],
            value=1,
            description='Axe 2:',
            disabled=False,
)


s_year1 = widgets.IntSlider(min=2015,max=2022,step=1, value = 2015)
s_year2 = widgets.IntSlider(min=2015,max=2022,step=1, value = 2018)


# X2 = pd.DataFrame(y_release).copy()
# X2.reset_index(inplace = True)


def graph_year (y_1,y_2,ax1,ax2):
    NewVar = var+'_cat'
    
    if y_1<y_2:
        if y_1 == 30:
            X2[NewVar] = ['sup'+ str(y_2) if (x >= y_2) else 'inf'+ str(y_2) for x in X2[var]]
        else:
            X2[NewVar] = ['sup'+ str(y_2) if (x >= y_2) else 'inf'+ str(y_1) if x < y_1 else 'neutre'for x in X2[var]]
        coord2 = pd.DataFrame(coord)
        coord2 = coord2.join(X2)

        import plotly.express as px
        fig = px.scatter(coord2, x=ax1, y=ax2, color=NewVar, hover_name='Batch')
        fig.add_trace(go.Scatter(x=[-8,8], y=[0,0], mode='lines', line_color = 'rgb(192,192,192)',showlegend=False))
        fig.add_trace(go.Scatter(x=[0,0], y=[-8,8], mode='lines', line_color = 'rgb(192,192,192)',showlegend=False))
        fig.update_xaxes(range=[-8,8])
        fig.update_yaxes(range=[-8,8])

        fig.update_layout(newshape_line=dict(color =('red')))

        fig.update_layout(title='Taux disso 30mn')
        fig.update_layout(title_x=0.5)
      #  fig.update_layout(width=900, height=600)
        fig.show()
    
    else:
        print('y_1 doit inféririeur à y_2')

    return 

interact(graph_year,y_1=s_year1, y_2 = s_year2, ax1 = wax1, ax2=wax2)

# Positionnement des lots en fonction d'un paramètre

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px



def graph_feature(f):
    
    fig = make_subplots(rows=1, cols=2)
    
    fig.add_trace(
        go.Scatter(x=X.index, y=X[f],name = 'Evolution'),#showlegend=False),
        row=1, col=1
    )
   
    fig.add_trace(
        go.Histogram(x=X[f],name = 'Distribution'),#showlegend=False),
        row=1, col=2
    )     
#    fig.update_layout(height=700, width=900, title_text= f)
    fig.update_layout(title_x=0.5)
    fig.update_layout(legend=dict(orientation="h",yanchor="top",y=1.1,xanchor="right",x=1))
    
    fig.show()
    
interact (graph_feature, f = dd_param)
#interact (graph_feature2, f = dd_param)


In [ ]:
seuil1b = widgets.FloatSlider()
seuil2b = widgets.FloatSlider(value = 0.1, step =0.001)


def slider_update(change):
    
    mi = pd.DataFrame(X.min()).to_dict()[0]
    ma = pd.DataFrame(X.max()+1).to_dict()[0]

    seuil1b.min = 0
    seuil1b.value = 0
    seuil1b.max = ma[dd_param.value]
    
    seuil2b.min = 0
    seuil2b.value = mi[dd_param.value]
    seuil2b.max = ma[dd_param.value]
    if ma[dd_param.value] < 0.2:
        seuil2b.step = 0.01
        
    
    

def graph_feature (f,ax1,ax2,s1,s2):
    
    X2c = pd.DataFrame(X[f]).copy()
    X2c.reset_index(inplace = True)
    NewVar = f+'_cat' 
    
    if s1<s2:
        if s1 == 0:
            X2c[NewVar] = ['sup'+ str(s2) if x >s2 else 'inf'+ str(s2) for x in X2c[f]]
        else:
            X2c[NewVar] = ['sup'+ str(s2) if x > s2 else 'inf'+ str(s1) if x < s1 else 'neutre'for x in X2c[f]]
        coord2 = pd.DataFrame(coord)
        coord2 = coord2.join(X2c)

        import plotly.express as px
        fig = px.scatter(coord2, x=ax1, y=ax2, color=NewVar, hover_name='Batch')
        fig.add_trace(go.Scatter(x=[-8,8], y=[0,0], mode='lines', line_color = 'rgb(192,192,192)',showlegend=False))
        fig.add_trace(go.Scatter(x=[0,0], y=[-8,8], mode='lines', line_color = 'rgb(192,192,192)',showlegend=False))
        fig.update_xaxes(range=[-8,8])
        fig.update_yaxes(range=[-8,8])

        #fig.update_layout(newshape_line=dict(color =('red')))
        fig.update_layout(legend=dict(orientation="v",yanchor="top",y=1.07,xanchor="right",x=1))
        fig.update_layout(title=f)
        fig.update_layout(title_x=0.5)
    #    fig.update_layout(width=900, height=500)
        fig.show()
    
    else:
        print('s1 doit inféririeur à s2')
    
    return  

dd_param.observe(slider_update, names='value')
interact (graph_feature, f = dd_param,  ax1 = wax1, ax2 = wax2,s1 = seuil1b, s2=seuil2b)

In [ ]:
#Xt = pd.DataFrame (coord)[[0,1]]
Xt = pd.DataFrame (coord)
#Xt = pd.DataFrame(X['Temperature_sortie_REFROIDISSEMENT']-X['Temperature_entree_REFROIDISSEMENT'])
#Xt = pd.DataFrame(X[['Granulometrie: d(50)-MEAN', 'Granulometrie: d(90)-MEAN']])
#Xt = pd.DataFrame (coord)[[0,1,3,11,13,19,23,26,27,55]]
y = pd.DataFrame(y_release)

y_mov = y.rolling(10,min_periods=1).mean() 


from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score




from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(Xt, y_mov, test_size=0.2, random_state=42)
# print('X_train: ', X_train.shape)
# print('X_val: ', X_val.shape)
# print('y_train: ', y_train.shape)
# print('y_val: ', y_val.shape)
# print()

# # Create linear regression object
model = linear_model.LinearRegression()
model.fit(X_train,y_train)
# print('Coefficients: \n', model.coef_)
# print()

# #Xgboost
# import xgboost as xgb
# model = xgb.XGBRegressor()
# model.fit(X_train, y_train)


# #SVR
# from sklearn.svm import SVR
# model = SVR(kernel = 'linear')
# model.fit(X_train, y_train)

# Make predictions using the testing set
y_val_pred = model.predict(X_val)
y_train_pred = model.predict(X_train)

# The mean squared error

# print ('***train***')
# print('Mean squared error on train: {:.2f}, sqrt: {:.2f}'.format(mean_squared_error(y_train, y_train_pred),
#                                                            np.sqrt(mean_squared_error(y_train, y_train_pred))
#                                                           )
# )
      
# print('R² : %.2f'
#       % r2_score(y_train, y_train_pred))

# print()
# print ('***val***')
# # The mean squared error
# print('Mean squared error: {:.2f}, sqrt: {:.2f}'.format(mean_squared_error(y_val, y_val_pred),
#                                                         np.sqrt(mean_squared_error(y_val, y_val_pred))
#                                                 )
#      )
# print('R² on Val: %.2f'
#       % r2_score(y_val, y_val_pred))


# Plot outputs
#plt.scatter(Xt, y,  color='black')
#plt.plot(Xt, y_pred, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())

#plt.show()

In [ ]:
dim_reg =  widgets.Dropdown(
    options=[i for i in range(1,len(X.columns))],
    value=45,
    description='Dimension:',
    disabled=False,
)

c_act= widgets.Checkbox(
    value=True,
    description='Disso 30mn',
    disabled=False,
    indent=False
)


def graph_disso30mn (d, c): 
    
    Xx = pd.DataFrame (coord)[[x for x in range(0,d)]]
    y = pd.DataFrame(y_release)
    model.fit(Xx,y)
    y_pred = model.predict(Xx)
    fig = make_subplots(rows=1, cols=1)
    
    if c:
        fig.add_trace(go.Scatter(x=y.index, y=y['RELEASE'], name = 'Disso 30mn'))
        fig.add_trace(go.Scatter(x=y.index, y=pd.DataFrame(y_pred)[0], name ='Disso 30 en fonction des Process, MP, PA'))
        fig.add_trace(go.Scatter(x=y.index, y=y_mov['RELEASE'], name ='Disso 30mn en moyenne mobile'))
    else:
 
        fig.add_trace(go.Scatter(x=y.index, y=pd.DataFrame(y_pred)[0], name ='Disso 30 en fonction des Process, MP, PA'))
        fig.add_trace(go.Scatter(x=y.index, y=y_mov['RELEASE'], name ='Disso 30mn en moyenne mobile'))

   # fig.update_layout(height=400, width=900, title_text= 'Reconstruction du taux de Dissolution 30mn (DRONEDARONE)')
    fig.update_layout( title_text= 'Reconstruction du taux de dissolution à 30mn (DRONEDARONE)')
    fig.update_layout(title_x=0.5)
    fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=-0.3,xanchor="right",x=1))
    #fig.update_xaxes(range=[-8,8])
    #fig.update_yaxes(range=[35,50])
    
    fig.show()
    y1 = y_mov.reset_index(drop = True).copy()
    y1.columns = ['mov']
    y2=pd.DataFrame(y_pred)
    y2.columns = ['pred']
    z=y1.join(y2)
    print ('Correlation: {:.2f}%'.format(z.corr().iloc[0,1]))
    
interact (graph_disso30mn, d=dim_reg, c= c_act)
    